In [1]:
import pandas as pd
import re
import math
import numpy as np

## Preprocessing

In [2]:
shanghaiDataset = "https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/shanghai.csv"
timesDataset = "https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/times.csv"
worldDataset="https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/world.csv"
educational_attainment_supplementary_data ="https://raw.githubusercontent.com/riiduan/ProgettoFondamentiDiInformatica/main/data/educational_attainment_supplementary_data.csv"

In [3]:
#funzione che legge file csv e restituisce pandas dataframe
def readDataSet(url , sep =',' , encoding='utf8',header='infer'):
    return pd.read_csv(url,sep=sep ,header=header,encoding=encoding)

In [4]:
# leggo i dataset
shanghai_df=readDataSet(shanghaiDataset)
times_df=readDataSet(timesDataset)
world_df=readDataSet(worldDataset)

In [5]:
# tolgo le rige dove nome di universita è null

index =shanghai_df[shanghai_df['university_name'].isnull()==True]
shanghai_df.drop(index.index,inplace = True)
shanghai_df.reset_index(drop=True, inplace=True)

In [6]:
# rinomino institutution in "university_name" per averla uguale come i altri dataframe

world_df.rename(columns={'institution':'university_name'},inplace=True)

In [7]:
shanghai_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       4896 non-null   object 
 1   university_name  4896 non-null   object 
 2   national_rank    4896 non-null   object 
 3   total_score      1101 non-null   float64
 4   alumni           4896 non-null   float64
 5   award            4895 non-null   float64
 6   hici             4895 non-null   float64
 7   ns               4875 non-null   float64
 8   pub              4895 non-null   float64
 9   pcp              4895 non-null   float64
 10  year             4896 non-null   int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 420.9+ KB


#### controllo le informazioni riguarda ogni dataframe per avere un visione di quanto rige ci sono e i tipo di colone

In [8]:
times_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [9]:
world_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   university_name       2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [10]:
'''
questo funzione prende come input il dataframe,  il flag ,e di default nome di universita ,anno,
e in base se è max faccio groupby nome di universita e in base a anno prendo idmax altrimenti idxmin
 

'''
def extract_dataset_min_max(dataframe, flag ,university_name='university_name',year='year' ):
    '''
    find most recent or least recent of data
    @dataframe : dataframe in input
    @university_name  : column university name 
    @year : column year
    @index : max or min
    
    '''
    dataset = pd.DataFrame()
    if(flag == 'max'):
          dataset = dataframe.iloc[dataframe.groupby(university_name)[year].idxmax()].copy()
            
    else: dataset = dataframe.iloc[dataframe.groupby(university_name)[year].idxmin()].copy()
    return dataset

In [11]:
times_most_recent_data = extract_dataset_min_max(times_df,'max')

In [12]:
times_least_recent_data = extract_dataset_min_max(times_df,'min')

In [13]:
times_most_recent_data[['world_rank','university_name','year']].head(10)

,world_rank,university_name,year
2405,601-800,AGH University of Science and Technology,2016
2003,201-250,Aalborg University,2016
2056,251-300,Aalto University,2016
1908,106,Aarhus University,2016
2105,301-350,Aberystwyth University,2016
2404,601-800,Adam Mickiewicz University,2016
2057,251-300,Aix-Marseille University,2016
2406,601-800,Ajou University,2016
2408,601-800,Alexandria University,2016
2409,601-800,Alexandru Ioan Cuza University,2016


In [14]:
times_least_recent_data[['world_rank','university_name','year']].head(10)

,world_rank,university_name,year
2405,601-800,AGH University of Science and Technology,2016
501,301-350,Aalborg University,2012
502,301-350,Aalto University,2012
166,167,Aarhus University,2011
476,276-300,Aberystwyth University,2012
2404,601-800,Adam Mickiewicz University,2016
2057,251-300,Aix-Marseille University,2016
2406,601-800,Ajou University,2016
146,147,Alexandria University,2011
2409,601-800,Alexandru Ioan Cuza University,2016



## 2. For each university, compute the improvement in income between the least recent and the most recent data points

nota : la colona "income" contiene valori "-" , percui prosego prima alla pulizia e poi lo converto in float

In [15]:
'''
In questa funzione filtro le rige dove la colona income coniente il charatere '-' , prendo tre colone che mi intressano
e converto la collona income in tipo float
'''
def clean_column_income(dataset):
    dataset=dataset[dataset.income.str.contains('-') ==False][['university_name','income','year']].copy()
    dataset.income=dataset.income.astype(float)
    return dataset
            

In [16]:
times_least_recent_data_clean = clean_column_income(times_least_recent_data)

In [17]:
times_most_recent_data_clean= clean_column_income(times_most_recent_data)

Unisco i due dataset tramite la colona "university_name"

In [18]:
union_most_and_least_data =pd.merge(times_least_recent_data_clean,times_most_recent_data_clean,on='university_name')

rinomino le colone 

In [19]:
union_most_and_least_data.rename(columns={'income_x':'income_level_least_recent','year_x':'year_least_recent','income_y':'income_level_most_recent','year_y':'year_most_recent'},inplace=True)
union_most_and_least_data

,university_name,income_level_least_recent,year_least_recent,income_level_most_recent,year_most_recent
0,Aalborg University,36.4,2012,43.7,2016
1,Aalto University,61.9,2012,61.6,2016
2,Aarhus University,61.5,2011,68.3,2016
3,Aberystwyth University,35.5,2012,31.3,2016
4,Adam Mickiewicz University,28.7,2016,28.7,2016
...,...,...,...,...,...
699,Zhejiang University,70.3,2011,96.2,2016
700,École Normale Supérieure,30.7,2011,37.1,2016
701,École Normale Supérieure de Lyon,26.1,2011,31.7,2016
702,École Polytechnique Fédérale de Lausanne,38.0,2011,65.4,2016


##### nota : alcuni universita hanno come "year_most_recent" e "year_most_recent" uguale percui non ha senso tenerli

In [20]:
union_most_and_least_data[union_most_and_least_data['year_least_recent']==union_most_and_least_data['year_most_recent']].head(100)

,university_name,income_level_least_recent,year_least_recent,income_level_most_recent,year_most_recent
4,Adam Mickiewicz University,28.7,2016,28.7,2016
5,Aix-Marseille University,33.1,2016,33.1,2016
6,Ajou University,45.7,2016,45.7,2016
8,Alexandru Ioan Cuza University,28.2,2016,28.2,2016
9,Aligarh Muslim University,29.6,2016,29.6,2016
...,...,...,...,...,...
179,Khon Kaen University,35.7,2016,35.7,2016
181,King Fahd University of Petroleum and Minerals,81.2,2016,81.2,2016
184,Kingston University,28.5,2016,28.5,2016
186,Kinki University,33.2,2016,33.2,2016


elimino queste colone

In [21]:
union_most_and_least_data.drop(union_most_and_least_data[union_most_and_least_data['year_least_recent']==union_most_and_least_data['year_most_recent']].index,inplace=True)

calcolo la diferenza

In [22]:
union_most_and_least_data["diference_income"] = union_most_and_least_data.income_level_most_recent - union_most_and_least_data.income_level_least_recent

creo una colona "improvement" che mi indica UP o Down in base se la income è positiva o negativa

In [23]:
union_most_and_least_data["improvement"] = np.where(union_most_and_least_data["diference_income"] >= 0, "UP", "Down")

In [24]:
union_most_and_least_data.head(5)

,university_name,income_level_least_recent,year_least_recent,income_level_most_recent,year_most_recent,diference_income,improvement
0,Aalborg University,36.4,2012,43.7,2016,7.3,UP
1,Aalto University,61.9,2012,61.6,2016,-0.3,Down
2,Aarhus University,61.5,2011,68.3,2016,6.8,UP
3,Aberystwyth University,35.5,2012,31.3,2016,-4.2,Down
7,Alexandria University,36.0,2011,29.7,2016,-6.3,Down


## 3. Find the university with the largest increase computed in the previous point

In [25]:
# filtro la colona diference_income dal dataset sopra che ha valore piu alto
union_most_and_least_data[union_most_and_least_data.diference_income ==union_most_and_least_data.diference_income.max()]

,university_name,income_level_least_recent,year_least_recent,income_level_most_recent,year_most_recent,diference_income,improvement
373,TU Dresden,31.9,2012,99.7,2016,67.8,UP


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

#### preprocessing 

nota: dataset shanghai_df , times_df ha alcuni valori espressi in range

In [26]:
shanghai_df[shanghai_df['world_rank'].str.contains('-')==True]['world_rank'].head(3)

100    101-152
101    101-152
102    101-152
Name: world_rank, dtype: object

nota : time_df porta con se alcuni valri con segno '=' davanti, che significa valore uguale, percui se lo tolgo non perde significato

In [27]:
times_df[times_df['world_rank'].str.contains('=')==True]['world_rank'].head(3)

Series([], Name: world_rank, dtype: object)

In [28]:
times_df['world_rank']=times_df['world_rank'].str.replace('=','')

nota: world_df sembra ok , non ha valori in range e valori strani

In [29]:
world_df['world_rank'].dtype

dtype('int64')

 ### Dati più recenti per le tabelle 'shanghai_ranking', 'times_ranking' e 'world_ranking

Shanghai

In [30]:
shanghai_most_recent_data = extract_dataset_min_max(shanghai_df,'max')

In [31]:
shanghai_most_recent_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658 entries, 4696 to 4545
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       658 non-null    object 
 1   university_name  658 non-null    object 
 2   national_rank    658 non-null    object 
 3   total_score      112 non-null    float64
 4   alumni           658 non-null    float64
 5   award            658 non-null    float64
 6   hici             658 non-null    float64
 7   ns               655 non-null    float64
 8   pub              658 non-null    float64
 9   pcp              658 non-null    float64
 10  year             658 non-null    int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 61.7+ KB


Times

In [32]:
times_most_recent_data = extract_dataset_min_max(times_df,'max')

In [33]:
times_most_recent_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 2405 to 2134
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              818 non-null    object 
 1   university_name         818 non-null    object 
 2   country                 818 non-null    object 
 3   teaching                818 non-null    float64
 4   international           818 non-null    object 
 5   research                818 non-null    float64
 6   citations               818 non-null    float64
 7   income                  818 non-null    object 
 8   total_score             818 non-null    object 
 9   num_students            796 non-null    object 
 10  student_staff_ratio     796 non-null    float64
 11  international_students  793 non-null    object 
 12  female_male_ratio       742 non-null    object 
 13  year                    818 non-null    int64  
dtypes: float64(4), int64(1), object(9)
mem

World  

In [34]:
world_most_recent_data = extract_dataset_min_max(world_df,"max")

In [35]:
world_most_recent_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 1981 to 1838
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            1024 non-null   int64  
 1   university_name       1024 non-null   object 
 2   country               1024 non-null   object 
 3   national_rank         1024 non-null   int64  
 4   quality_of_education  1024 non-null   int64  
 5   alumni_employment     1024 non-null   int64  
 6   quality_of_faculty    1024 non-null   int64  
 7   publications          1024 non-null   int64  
 8   influence             1024 non-null   int64  
 9   citations             1024 non-null   int64  
 10  broad_impact          1023 non-null   float64
 11  patents               1024 non-null   int64  
 12  score                 1024 non-null   float64
 13  year                  1024 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 120.0+ KB


### Creo nuova colona con nomi di universita puliti (spazi,virgole,tutti minuscoli ect) cosi combaciano

In [36]:
def cleanNames(name):
    cleanname =name.lower().replace("'", "").replace(" ", "").replace(",", "").replace("-", "")
    return cleanname

In [37]:
shanghai_most_recent_data['CleanUniNames'] = shanghai_most_recent_data.university_name.apply(cleanNames)

In [38]:
times_most_recent_data['CleanUniNames'] = times_most_recent_data.university_name.apply(cleanNames)


In [39]:
world_most_recent_data['CleanUniNames'] = world_most_recent_data.university_name.apply(cleanNames)

### Prendo solo le colone che mi interessano

In [40]:
shanghai_sh=shanghai_most_recent_data[['CleanUniNames','university_name','world_rank']].rename(columns={"world_rank" : "world_rank_shanghai"})

In [41]:
print(shanghai_sh)

               CleanUniNames           university_name world_rank_shanghai
4696       aalborguniversity        Aalborg University             301-400
4796         aaltouniversity          Aalto University             401-500
4468        aarhusuniversity         Aarhus University                  73
4496  aixmarseilleuniversity  Aix Marseille University             101-150
3115  aixmarseilleuniversity  Aix-Marseille University             102-150
...                      ...                       ...                 ...
3512     yamaguchiuniversity      Yamaguchi University             401-500
4694       yeshivauniversity        Yeshiva University             201-300
4695        yonseiuniversity         Yonsei University             201-300
4395          yorkuniversity           York University             401-500
4545      zhejianguniversity       Zhejiang University             101-150

[658 rows x 3 columns]


In [42]:
times_sh=times_most_recent_data[['CleanUniNames','world_rank']].rename(columns={"world_rank" : "world_rank_times"})

In [43]:
world_sh=world_most_recent_data[['CleanUniNames','world_rank']].rename(columns={"world_rank" : "world_rank_world"})

### Inner join, prendo solo i nomi in comune

In [44]:
JoinTableFirst = shanghai_sh.join(times_sh.set_index('CleanUniNames'), on='CleanUniNames', how='inner')

In [45]:
JoinTable = JoinTableFirst.join(world_sh.set_index('CleanUniNames'), on='CleanUniNames', how='inner')

In [46]:
JoinTable

,CleanUniNames,university_name,world_rank_shanghai,world_rank_times,world_rank_world
4696,aalborguniversity,Aalborg University,301-400,201-250,565
4796,aaltouniversity,Aalto University,401-500,251-300,421
4468,aarhusuniversity,Aarhus University,73,106,122
4496,aixmarseilleuniversity,Aix Marseille University,101-150,251-300,206
3115,aixmarseilleuniversity,Aix-Marseille University,102-150,251-300,206
...,...,...,...,...,...
4406,yaleuniversity,Yale University,11,12,11
4694,yeshivauniversity,Yeshiva University,201-300,164,171
4695,yonseiuniversity,Yonsei University,201-300,301-350,98
4395,yorkuniversity,York University,401-500,301-350,337


### divido i range in due colone

In [47]:
def splitRange(columnValue):
      result=columnValue.split('-')
      if len(result) >1 :
                return [int(result[0]),int(result[1])]
      else:
                return [int(result[0]),int(result[0])]

In [48]:
Mylist =[]
def FindDiff(col1,col2,col3):
    
    Mylist.extend(splitRange(col1))
    Mylist.extend(splitRange(col2))
    Mylist.extend(splitRange(str(col3)))
    Mylist.sort()
    min1 = int(Mylist[0])
    max1 =int(Mylist[-1])
    Mylist.clear()
    return max1-min1

In [49]:
JoinTable["MaxDiff"]=JoinTable.apply(lambda x: FindDiff(x.world_rank_shanghai, x.world_rank_times,x.world_rank_world), axis=1 )

In [50]:
FinalTable = JoinTable[['university_name','world_rank_shanghai','world_rank_times', 'world_rank_world', 'MaxDiff']]

In [51]:
FinalTable.head(5)

,university_name,world_rank_shanghai,world_rank_times,world_rank_world,MaxDiff
4696,Aalborg University,301-400,201-250,565,364
4796,Aalto University,401-500,251-300,421,249
4468,Aarhus University,73,106,122,49
4496,Aix Marseille University,101-150,251-300,206,199
3115,Aix-Marseille University,102-150,251-300,206,198



## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [52]:
times_most_recent_data.num_students

2405    35,569
2003    17,422
2056    16,099
1908    23,895
2105     9,252
         ...  
1856     2,400
2013     2,218
1904     2,429
1833     9,666
2134     8,788
Name: num_students, Length: 818, dtype: object

In [53]:
times_most_recent_data.num_students.hasnans

True

#### Nota: vedo che la colona num_students usa la virgoa come seperatore di mille , percui gli tolgo

In [54]:
times_most_recent_data['num_students']=times_most_recent_data.num_students.str.replace(',','').astype(float)

#### Controllo la collona female_male_ratio

In [55]:
times_most_recent_data.female_male_ratio.hasnans

True

In [56]:
times_most_recent_data.female_male_ratio.values

array(['-', '48:52:00', '32 : 68', '54:46:00', '48:52:00', '71:29:00',
       '61:39:00', '33 : 67', '46:54:00', '63:37:00', '0.765972222',
       '60:40:00', '51:49:00', '48:52:00', '34 : 66', nan, '78:22:00',
       '36 : 64', '52:48:00', '50:50:00', '53:47:00', nan, '47:53:00',
       '51:49:00', '58:42:00', '47:53:00', '52:48:00', '59:41:00',
       '56:44:00', '67:33:00', '55:45:00', '59:41:00', '1.011805556',
       '46:54:00', nan, '69:31:00', '49:51:00', '57:43:00', nan,
       '48:52:00', '55:45:00', '0.970833333', '55:45:00', '54:46:00',
       '58:42:00', '53:47:00', '50:50:00', '55:45:00', '27 : 73', nan,
       '46:54:00', '32 : 68', '47:53:00', '33 : 67', '58:42:00',
       '70:30:00', '57:43:00', '47:53:00', '43:57:00', '39 : 61',
       '48:52:00', nan, '66:34:00', '59:41:00', '30 : 70', '42:58:00',
       '64:36:00', '67:33:00', '62:38:00', '0.847916667', '37 : 63',
       '57:43:00', '53:47:00', nan, nan, '53:47:00', '45:55:00', nan,
       '41:59:00', '59:41:00', '41

#### Vedo che ha valori  "-" e valori "nan" , percui gli devo ignorare

In [57]:
times_most_recent_data.female_male_ratio

2405              -
2003       48:52:00
2056        32 : 68
1908       54:46:00
2105       48:52:00
           ...     
1856       46:54:00
2013       49:51:00
1904    0.806944444
1833        27 : 73
2134       61:39:00
Name: female_male_ratio, Length: 818, dtype: object

In [58]:
times_most_recent_data['first_condition'] = times_most_recent_data['female_male_ratio'].str.contains('-')==True
times_most_recent_data['second_condition'] = times_most_recent_data['female_male_ratio'].isnull()
times_most_recent_data['third_condition'] = times_most_recent_data['num_students'].isnull()

### Costruisco due colone uno per numeri di maschi e uno per numero di femmine

In [59]:
def CalculateFemale(row):
    if( row["first_condition"] | row["second_condition"] | row["third_condition"]):
        return None
    
    femaleratio = row["female_male_ratio"].split(':')[0]
    totalnrStudents = row["num_students"]
    return round((femaleratio/100) *totalnrStudents,0)
    

In [60]:
times_most_recent_data

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,CleanUniNames,first_condition,second_condition,third_condition
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,-,2016,aghuniversityofscienceandtechnology,True,False,False
2003,201-250,Aalborg University,Denmark,25.1,71,28.4,73.8,43.7,-,17422.0,15.9,15%,48:52:00,2016,aalborguniversity,False,False,False
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,aaltouniversity,False,False,False
1908,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54:46:00,2016,aarhusuniversity,False,False,False
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48:52:00,2016,aberystwythuniversity,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69,2400.0,7.9,20%,46:54:00,2016,écolenormalesupérieure,False,False,False
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49:51:00,2016,écolenormalesupérieuredelyon,False,False,False
1904,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,0.806944444,2016,écolepolytechnique,False,False,False
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016,écolepolytechniquefédéraledelausanne,False,False,False


In [61]:
times_most_recent_data["nr_female"]= times_most_recent_data.apply(CalculateFemale , axis=1)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Il numero di maschi è la differenza tra numero totale e numero di femmine
times_most_recent_data["nr_male"]=times_most_recent_data.num_students - times_most_recent_data.nr_female

In [ ]:
times_most_recent_data[["country","nr_female","nr_male","num_students","female_male_ratio"]]

In [ ]:
lista =["nr_female","nr_male","num_students"]

In [ ]:
# raggruppo per country e faccio la somma

female_male_df = times_most_recent_data.groupby('country', as_index=False)[lista].sum()

In [ ]:
female_male_df.tail(16)

### Note : Vedo che 2 nomi di universita sono scritti male percui gli elimino

In [ ]:
female_male_df.drop([67,71], inplace=True)

## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [ ]:
def Calcolate_ratio(row):
    if math.isnan(row["nr_male"]) or math.isnan(row["nr_female"]) or row["nr_male"] == 0:
        return None
    else:
        return round((row["nr_female"]/row["nr_male"])*100)

In [ ]:
times_most_recent_data["Ratio"] = times_most_recent_data.apply(Calcolate_ratio,axis=1)

In [ ]:
times_most_recent_data.Ratio

In [ ]:
avarage_ratio = round(times_most_recent_data.Ratio.mean(),2)

In [ ]:
avarage_ratio

In [ ]:
female_male_below_ratio = times_most_recent_data[times_most_recent_data.Ratio < avarage_ratio]

In [ ]:
female_male_below_ratio[["university_name","Ratio"]]

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [ ]:
# Number of total student for country i have it from "female_male_df"
female_male_df.head()

In [ ]:
list_of_columns=["num_students","country"]
students_below_ratio=female_male_below_ratio.groupby('country', as_index=False)[list_of_columns].sum()

In [ ]:
ratio_below_and_total = pd.merge(female_male_df, students_below_ratio, how="inner", on=["country"])
ratio_below_and_total["Ratio"] = round((ratio_below_and_total.num_students_y/ratio_below_and_total.num_students_x)*100,2)

In [ ]:
#rename "num_students_x" to "num_students_total"  and "num_students_y" to "num_students_below"
ratio_below_and_total = ratio_below_and_total.rename(columns={'num_students_x': 'num_students_total', 'num_students_y': 'num_students_below'})

In [ ]:
ratio_below_and_total.head(10)

## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment_supplementary =readDataSet(educational_attainment_supplementary_data)

In [ ]:
index_to_drop=educational_attainment_supplementary[(educational_attainment_supplementary.country_name.isnull() ==True) | (educational_attainment_supplementary.series_name.isnull() ==True) ].index.values

In [ ]:
educational_attainment_supplementary.drop(index_to_drop, inplace=True)


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
value_vars= ['1985', '1986', '1987', '1990', '1991',
       '1992', '1993', '1995', '1996', '1997', '1998', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2015']
id_vars=['country_name', 'series_name']

In [ ]:
unpivot_attainment=pd.melt(educational_attainment_supplementary,id_vars=id_vars , value_vars=value_vars, var_name='year', value_name='value')

In [ ]:
unpivot_attainment.head(5)

## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

### nota : normalizo i nomi di universita utilizando la funzione cleanNames

In [ ]:
times_df["uni_name_normalazed"] = times_df.university_name.apply(cleanNames)

In [ ]:
shanghai_df["uni_name_normalazed"]= shanghai_df.university_name.apply(cleanNames)

In [ ]:
world_df["uni_name_normalazed"] = world_df.university_name.apply(cleanNames)

In [ ]:
world_df.uni_name_normalazed

In [ ]:
### procedo a prendere i nomi di universta per ciascun dataframe

In [ ]:
times_df

In [ ]:
times_uni = times_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
times_uni

In [ ]:
shanghai_uni = shanghai_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
world_uni = world_df[["university_name","uni_name_normalazed"]].groupby("uni_name_normalazed").max()

In [ ]:
all_uni = times_uni.append(shanghai_uni).append(world_uni)

In [ ]:
all_uni_group=all_uni.groupby("uni_name_normalazed").count()

In [ ]:
#i want to take back normal university name so i make a join
all_uni_count_merge = pd.merge(all_uni,all_uni_group, on="uni_name_normalazed")

In [ ]:
# i have duplicates so i remove them, take off index, and rename all
all_uni_count=all_uni_count_merge.drop_duplicates().reset_index()\
                .drop("uni_name_normalazed" ,axis=1)\
                .rename(columns={'university_name_x': 'university_name','university_name_y':'count'}) 

In [ ]:
all_uni_count.head(5)

## 11. In the times ranking, compute the number of times each university appears

In [ ]:
times_count_uni=times_df.copy()

In [ ]:
times_count_uni_pc =times_count_uni[["university_name","uni_name_normalazed"]]\
                    .groupby("university_name" , as_index=False).count() \
                     .rename(columns={'uni_name_normalazed': 'count'}) 

In [ ]:
times_count_uni_pc.head(10)

## 12. Find the universities that appear at most twice in the times ranking.

In [ ]:
uni_most_twice = times_count_uni_pc[times_count_uni_pc['count'] >= 2]

In [ ]:
uni_most_twice.head(5)

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [ ]:
shanghai_no_range = shanghai_df[shanghai_df['world_rank'].str.contains('-')==False]

In [ ]:
times_no_range = times_df[times_df['world_rank'].str.contains('-')==False]

In [ ]:
shanghai_no_range =shanghai_no_range[["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
shanghai_no_range.world_rank = shanghai_no_range.world_rank.astype('int')

In [ ]:
times_no_range=times_no_range[["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
times_no_range.world_rank=times_no_range.world_rank.astype('int')

In [ ]:
worlddf=world_df[world_df["world_rank"].isnull() == False][["uni_name_normalazed","university_name","world_rank","year"]]

In [ ]:
first_union_df =pd.merge(shanghai_no_range,times_no_range,on=["uni_name_normalazed","year"])\
                            .drop("university_name_y",axis=1)\
                             .rename(columns={'university_name_x': 'university','world_rank_x':'shanghai_rank','world_rank_y':'times_rank'}) 

In [ ]:
first_union_df

In [ ]:
same_position = pd.merge(first_union_df,worlddf, on=["uni_name_normalazed","year"] )\
                        .drop(['university_name','uni_name_normalazed'],axis=1)

In [ ]:
result = same_position[(same_position['shanghai_rank'] == same_position['times_rank'])\
                              & (same_position['times_rank'] == same_position['world_rank']) ]

In [ ]:
result